# <div align="center">Faster R-CNN</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

***Fast R-CNN*** depends on an external region proposal method like selective search. However, those algorithms run on CPU and they are slow. In testing, Fast R-CNN takes 2.3 seconds to make a prediction in which 2 seconds are for generating 2000 ROIs.

In [1]:
feature_maps = process(image)
ROIs = region_proposal(image)         # Expensive!
for ROI in ROIs
    patch = roi_pooling(feature_maps, ROI)
    results = detector2(patch)

***Faster R-CNN*** adopts similar design as the Fast R-CNN except it ***replaces the region proposal method by an internal deep network*** and the ROIs are derived from the feature maps instead. ***The new region proposal network (RPN) is more efficient and run at 10 ms per image in generating ROIs.***

<img src='asset/7_4/1.png'>
<div align="center">Network flow is the same as the Fast R-CNN.</div>

The network flow is similar but the region proposal is now replaced by a convolutional network (***RPN***).

<img src='asset/7_4/2.jpg'>
<div align="center">More Precise Network flow for Faster R-CNN.</div>

# <div align="center">Region proposal network</div>
---------------------------------------------------------------------

The region proposal network (RPN) takes the output feature maps from the first convolutional network as input. It slides 3 × 3 filters over the feature maps to make class-agnostic region proposals using a convolutional network like ZF network (below). Other deep network likes VGG or ResNet can be used for more comprehensive feature extraction at the cost of speed. The ZF network outputs 256 values, which is feed into 2 separate fully connected layers to predict a boundary box and 2 objectness scores. The ***objectness measures whether the box contains an object***. We can use a regressor to compute a single objectness score but for simplicity, Faster R-CNN uses a classifier with 2 possible classes: one for the “have an object” category and one without (i.e. the background class).

<img src='asset/7_4/3.jpg'>
<div align="center">ZF Network structure</div>

***For each location in the feature maps, RPN makes k guesses***. Therefore RPN outputs 4×k coordinates and 2×k scores per location. The diagram below shows the 8 × 8 feature maps with a 3× 3 filter, and it outputs a total of ***8 × 8 × 3 ROIs*** (for k = 3). The right side diagram demonstrates the 3 proposals made by a single location.

<img src='asset/7_4/4.jpg'>

Here, we get 3 guesses and we will refine our guesses later. Since we just need one to be correct, we will be better off if our initial guesses have different shapes and size. Therefore, Faster R-CNN does not make random boundary box proposals. Instead, it predicts offsets like ***𝛿x, 𝛿y*** that are relative to the top left corner of some reference boxes called anchors. We constraints the value of those offsets so our guesses still resemble the anchors.

<img src='asset/7_4/5.png'>

To make k predictions per location, we need k anchors centered at each location. Each prediction is associated with a specific anchor but different locations share the same anchor shapes.

<img src='asset/7_4/6.png'>

Those anchors are carefully pre-selected so they are diverse and cover real-life objects at different scales and aspect ratios reasonable well. This guides the initial training with better guesses and allows each prediction to specialize in a certain shape. This strategy makes early training more stable and easier.

***Faster R-CNN uses far more anchors. It deploys 9 anchor boxes: 3 different scales at 3 different aspect ratio. Using 9 anchors per location, it generates 2 × 9 objectness scores and 4 × 9 coordinates per location.***

<img src='asset/7_4/7.png'>

***Anchors are also called priors or default boundary boxes in different papers.***

# Summing it all up

Fast R-CNN is much faster in both training and testing time. However, the improvement is not dramatic because the region proposals are generated separately by another model and that is very expensive.

An intuitive speedup solution is to integrate the region proposal algorithm into the CNN model. Faster R-CNN is doing exactly this: construct a single, unified model composed of RPN (region proposal network) and fast R-CNN with shared convolutional feature layers.

# <div align="center">Loss Function</div>
---------------------------------------------------------------------

<img src='asset/7_4/10.png'>